In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
import os
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

plt.style.use('dark_background')
np.set_printoptions(precision=3)

In [ ]:
log_path = 'data/log/'

In [ ]:
def get_results(folder_name: str, extractor):
    folder_path = f'{log_path}/{folder_name}'
    files = os.listdir(folder_path)
    res = []
    for file in files:
        with open(f'{folder_path}/{file}') as f:
            lines = f.readlines()
        res.append(extractor(lines))
    return res

def extractor(lines: [str], leading_str: str, target_i: int = -1):
    change_value = lines[0].split('-')[-1].strip()
    tar_line = next(l for l in lines if l.startswith(leading_str))
    tar = [c for c in tar_line.split(' ') if c][target_i]
    return change_value, float(tar)

In [ ]:
acc_ex = lambda l: extractor(l, 'Acc')
miou_ex = lambda l: extractor(l, 'mIoU')
ap50_ex = lambda l: extractor(l, 'average', -5)
bike_ap25 = lambda l: extractor(l, 'bike', -4)
building_ap25 = lambda l: extractor(l, 'bike', -4)

In [ ]:
def sort_array(a):
    a = a.T
    return a[a[:, 0].argsort()].T

In [ ]:
def plot(target, title, x_title, invert_x=False):
    r_ap50 = np.array(get_results(target, ap50_ex), dtype=float).T
    r_ap50 = sort_array(r_ap50)
    r_miou = np.array(get_results(target, miou_ex), dtype=float).T
    r_miou[1] = r_miou[1]/100
    r_miou = sort_array(r_miou)
    r_acc = np.array(get_results(target, acc_ex), dtype=float).T
    r_acc[1] = r_acc[1]/100
    r_acc = sort_array(r_acc)

    plt.plot(r_ap50[0], r_ap50[1], "-b", label="AP50")
    plt.plot(r_miou[0], r_miou[1], "-r", label="mIoU")
    plt.plot(r_acc[0], r_acc[1], "-g", label="Accuracy")
    plt.legend(loc="upper right")
    plt.ylim(0, 1.0)
    if invert_x:
        plt.xlim(1, 0)
    else:
        plt.xlim(0, 1)
    plt.title(title)
    plt.xlabel(x_title)
    img_name = f'images/{target}_performance_plot.png'
    plt.savefig(img_name, format='png', dpi=400)
    plt.show()

In [ ]:
plot('sample_random', 'Performance with decreasing random sample', 'Amount of points sampled', True)

In [ ]:
plot('sample_farthest', 'Performance with decreasing sample', 'Amount of points sampled', True)

In [ ]:
plot('noisy', 'Performance with add noise on cordinates', 'SD added to cordinates', False)

In [ ]:
np.array(get_results(target, acc_ex), dtype=float)

In [ ]:
np.array(get_results('sample_random', ap50_ex), dtype=float).T[0]

In [ ]:
get_results('sample_farthest', building_ap25)

In [ ]:
title = 'Furthest- and Random Sampling compared'
x_title = 'Amount of points sampled'
f_ap50 = np.array(get_results('sample_farthest', ap50_ex), dtype=float).T
f_ap50 = sort_array(f_ap50)
r_ap50 = np.array(get_results('sample_random', ap50_ex), dtype=float).T
r_ap50 = sort_array(r_ap50)


plt.plot(f_ap50[0], f_ap50[1], "-y", label="AP50 Furthest Sampling")
plt.plot(r_ap50[0], r_ap50[1], "-r", label="AP50 Random Sampling")
plt.legend(loc="upper right")
plt.ylim(0, 1.0)
if invert_x:
    plt.xlim(1, 0)
else:
    plt.xlim(0, 1)
plt.title(title)
plt.xlabel(x_title)
img_name = f'images/furthest_random_performance_plot.png'
plt.savefig(img_name, format='png', dpi=400)
plt.show()

In [ ]:
np.array(get_results('sample_random', ap50_ex))

In [ ]:
np.array(get_results('sample_farthest', ap50_ex))

In [ ]:
get_results('noisy', ap50_ex)

In [ ]:
CLASSES = ('floor', 'building', 'low vegetation', 'med. vegetation', 'high vegetation', 'vehicle',
               'truck', 'aircraft', 'militaryVehicle', 'bike', 'motorcycle', 'light pole',
               'street sign', 'clutter', 'fence')
target = 'no_color'


ap50_ex = lambda l: extractor(l, 'average', -5)

ap50s = []
for c in CLASSES[1:]:
    ex = lambda l: extractor(l, c, -5)
    t = get_results(target, ex)
    ap50s.append(next(p[1] for p in t if p[0] == '0'))
ap50s = [np.nan] + ap50s

In [ ]:
def ex(lines):
    tar_line = next(l for l in lines if l.startswith('Class-wise mIoU:'))
    return tar_line.split(':')[-1].strip().split(' ')
mious = get_results(target, ex)[-1]
mious = [float(i)/100 for i in mious]

In [ ]:
df = pd.DataFrame({'class': ('floor', 'building', 'low veg.', 'med. veg.', 'high veg.', 'vehicle',
               'truck', 'aircraft', 'militaryVeh.', 'bike', 'motorcycle', 'light pole',
               'street sign', 'clutter', 'fence'),
                   'AP50': ap50s,
                   'mIoU': mious})
df = df.set_index('class').transpose()
print(df.to_markdown())

In [ ]:
df